In [98]:
import refinitiv.data as rd
import pandas as pd
import datetime
import numpy as np
import cvxpy as cp
from dotenv import load_dotenv
import os
load_dotenv()

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3974773

In [100]:
session = rd.session.platform.Definition(
    app_key= os.getenv('REFINITIV_APP_KEY'),
    grant= rd.session.platform.GrantPassword(
        username=os.getenv('REFINITIV_USERNAME'),password=os.getenv('REFINITIV_PASSWORD')
    )
).get_session()
rd.session.set_default(session)

c:\Users\FWX\anaconda3\Lib\site-packages\refinitiv\data\session\platform.py:90: FutureWarning:


You open a platform session using the default value of the signon_control parameter (signon_control=True).
In future library version v2.0, this default will be changed to False.
If you want to keep the same behavior as today, you will need to set the signon_control parameter to True either in the library configuration file
({'sessions':{'platform':{'your_session_name':{'signon_control':true}}}}) or in your code where you create the Platform Session.
These alternative options are already supported in the current version of the library.



In [5]:
session.open()
thai_universe =rd.discovery.search(
	view = rd.discovery.Views.EQUITY_QUOTES,
    top = 887,
	filter = "(AssetState ne 'DC' and IsPrimaryRIC eq true and SearchAllCategoryv2 eq 'Equities' and (RCSAssetCategoryGenealogy in ('A:1L') and ExchangeName xeq 'The Stock Exchange of Thailand' and RCSCurrency xeq 'C:W'))",
	select = "DTSubjectName,ExchangeName,RIC"
)

In [6]:
list_thai_u = thai_universe['RIC'].tolist()
start_date = "2014-01-01"
end_date = "2024-01-01"

In [7]:
rd.open_session()
df = rd.get_history(
          universe = list_thai_u,
          fields = [          
               'TR.CO2EmissionTotal',
               'TR.F.EV',
               'TR.F.EBITDA'
          ], 
          start=start_date,
          end=end_date
)

c:\Users\FWX\anaconda3\Lib\site-packages\refinitiv\data\_access_layer\session.py:71:FutureWarning: 
You open a platform session using the default value of the signon_control parameter (signon_control=True).
In future library version v2.0, this default will be changed to False.
If you want to keep the same behavior as today, you will need to set the signon_control parameter to True either in the library configuration file
({'sessions':{'platform':{'your_session_name':{'signon_control':true}}}}) or in your code where you create the Platform Session.
These alternative options are already supported in the current version of the library.
c:\Users\FWX\anaconda3\Lib\site-packages\pandas\core\dtypes\cast.py:1079:RuntimeWarning: invalid value encountered in cast
c:\Users\FWX\anaconda3\Lib\site-packages\pandas\core\dtypes\cast.py:1104:RuntimeWarning: invalid value encountered in cast
c:\Users\FWX\anaconda3\Lib\site-packages\pandas\core\dtypes\cast.py:1079:RuntimeWarning: invalid value encountere

In [8]:
def reset_index_to_year(date:datetime.datetime)->int:
    return date.year

In [19]:
df_dict = dict()
for ticker in list_thai_u:
    if len(df[ticker].dropna())!=0:
        df_dict[ticker] = df[ticker].dropna().set_index([pd.Index(df[ticker].dropna().reset_index()['Date'].apply(reset_index_to_year).tolist())])

In [31]:
df_dict_optimize = dict()
for i in df_dict:
    df_dict_optimize[i] = pd.DataFrame(data={'EBITDA/EV':(df_dict[i]['Earnings before Interest Taxes Depreciation & Amortization']/df_dict[i]['Enterprise Value']).tolist(),
                                        'CO2/EV':(df_dict[i]['CO2 Equivalent Emissions Total']/df_dict[i]['Enterprise Value']).tolist()},index = df_dict[i].index.tolist())

In [90]:
y = 2017
Df_dict = dict()
for i in df_dict_optimize:
    if y in df_dict_optimize[i].index:
        Df_dict[i] = df_dict_optimize[i]

$$\begin{aligned}
	        \max_W \quad         & \cfrac{\textbf{EBITDA}}{\textbf{EV}}W^\top,\\
	        \textrm{subject to} \quad   & \cfrac{\textbf{CO2}}{\textbf{EV}}W^\top&\leq& \textbf{CO2budget},\\
			& w_i &\geq& 0 \\
	        \textrm{and} \quad          & W^\top\textbf{1}&=&1.
	\end{aligned}$$

In [91]:
EBITDA = np.array([Df_dict[i]['EBITDA/EV'][y] for i in Df_dict])
CO = np.array([Df_dict[i]['CO2/EV'][y] for i in Df_dict])

In [102]:
#create the decision variable x with lenth equals the length of mu
W = cp.Variable(len(EBITDA))
#define the required return (3 percent per month in this example)
r = 0.03

#define the objective function
Portfolio_Risk = EBITDA@W.T
Objective = cp.Maximize(Portfolio_Risk)

#define constraints
Co2_bud = CO@W.T
Co2_budget = 10**-3
Constraints = [Co2_bud <= Co2_budget,W>=0,sum(W)==1]

#solve the optimization problem
cp.Problem(Objective, Constraints).solve()

#extract the optimal portfolio
optimal_Portfolio = W.value

c:\Users\FWX\anaconda3\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning:


    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    



In [104]:
for i in optimal_Portfolio:
    print(i)

7.55362376055154e-11
8.642578032827607e-11
8.35206117689627e-11
8.419578691412314e-11
8.6661275822126e-11
0.999999997859397
8.46244847273083e-11
7.35652182960892e-11
7.380930980578698e-11
8.886168600742783e-11
7.563138740619719e-11
8.432469870702987e-11
9.132366211535779e-11
7.652025541349546e-11
7.19866596960851e-11
8.231381574671048e-11
8.724154534035217e-11
7.671057694509638e-11
7.394746446765082e-11
6.901977993321283e-11
5.683525661992096e-11
7.696922518257444e-11
6.815688538915219e-11
7.319529506898774e-11
7.792958313921125e-11
8.805406041697614e-11
8.19909996836052e-11
9.125793930855581e-11
